<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [1]:
import torch
import torchvision
import torchvision.transforms as transforms
import torchvision.datasets as datasets
# import pytorch_lightning as pl

import numpy as np
import random as rn
import pandas as pd

import matplotlib.pyplot as plt
import matplotlib

device = torch.device('cuda:1')

In [2]:
transform = transforms.Compose([transforms.Grayscale(num_output_channels=1),
                                transforms.ToTensor(),
                                transforms.Normalize((0.5), (0.5)),
                                transforms.CenterCrop(size=64)
                              ])

In [3]:
ds_all = datasets.ImageFolder('train', transform=transform)

In [4]:
ds_train, ds_val, _ = torch.utils.data.random_split(ds_all, [12000, 1400, 4])

In [5]:
trainloader = torch.utils.data.DataLoader(ds_train, batch_size=10,
                                        shuffle=True, num_workers=2)

valloader = torch.utils.data.DataLoader(ds_val, batch_size=10,
                                        shuffle=True, num_workers=2)

In [10]:
import torch.nn as nn
import torch.nn.functional as F

class ConvBlock(nn.Module):
    def __init__(self,
                 in_channels,     # <== number of input channels to the 1st convolution
                 interm_channels, # <== outputs of the 1st / inputs of the 2nd convolution
                 out_channels,    # <== outputs of the 2nd convolution
                 kernel_size,     #
                 pool_size,       
                 use_batchnorm,   # <== whether we'll use batchnorm
                 initialization): # <== function that'll initialize the weights
        
        # First we run the base class constructor
        super(ConvBlock, self).__init__()

        # And then define all the layers used within a block
        self.conv1 = nn.Conv2d(in_channels=in_channels,
                               out_channels=interm_channels,
                               kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(in_channels=interm_channels,
                               out_channels=out_channels,
                               kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(pool_size)

        self.use_batchnorm = use_batchnorm
        if use_batchnorm:
            self.bn1 = nn.BatchNorm2d(interm_channels)
            self.bn2 = nn.BatchNorm2d(out_channels)

        # If initialization function provided, call it on the weights of the model
        if initialization is not None:
            initialization(self.conv1.weight)
            initialization(self.conv2.weight)

    def forward(self, x):
        x = self.conv1(x)
        if self.use_batchnorm:
            x = self.bn1(x)
        x = F.relu(x)

        x = self.conv2(x)
        if self.use_batchnorm:
            x = self.bn2(x)
        x = F.relu(x)

        x = self.pool(x)
        return x
        
        
# The model itself:
class model(nn.Module):
    def __init__(self, use_batchnorm, initialization, dropout):
        super(model, self).__init__()

        # Convolutional layers:                                                     #576x576
#         self.conv1 = ConvBlock(1, 8, 8, 7,  3, use_batchnorm, initialization)      #192x192
#         self.conv2 = ConvBlock(8, 8, 16, 5, 3, use_batchnorm, initialization)     #64x64
#         self.conv3 = ConvBlock(16, 16, 32, 3, 2, use_batchnorm, initialization)     #32x32     
#         self.conv4 = ConvBlock(32, 32, 64, 3, 2, use_batchnorm, initialization)    #16x16
#         self.conv5 = ConvBlock(64, 64, 64, 3, 2, use_batchnorm, initialization)    #8x8
#         self.conv6 = ConvBlock(64, 64, 64, 1, 2, use_batchnorm, initialization)  #4x4
                                                                                    #64x64
        self.conv1 = ConvBlock(1, 8, 8, 4,  3, use_batchnorm, initialization)      #64x64
        self.conv2 = ConvBlock(8, 8, 16, 5, 2, use_batchnorm, initialization)     #32x32
        self.conv3 = ConvBlock(16, 16, 32, 3, 2, use_batchnorm, initialization)     #16x16     
        self.conv4 = ConvBlock(32, 32, 64, 3, 2, use_batchnorm, initialization)    #8x8
        self.conv5 = ConvBlock(64, 64, 64, 3, 2, use_batchnorm, initialization)    #4x4
        #self.conv6 = ConvBlock(64, 64, 64, 1, 2, use_batchnorm, initialization)  #2x2
        
    
    
        # Fully connected layers:
#         self.bn0 = nn.BatchNorm1d(64*4*4)
#         self.fc1 = nn.Linear(64*4*4, 64)
#         self.dropout1 = nn.Dropout(dropout)
#         self.bn1 = nn.BatchNorm1d(64)
#         self.fc2 = nn.Linear(64, 16)
#         self.dropout2 = nn.Dropout(dropout)
#         self.bn2 = nn.BatchNorm1d(16)
#         self.fc3 = nn.Linear(16, 2)
        self.bn0 = nn.BatchNorm1d(64)
        self.fc1 = nn.Linear(64, 32)
        self.dropout1 = nn.Dropout(dropout)
        self.bn1 = nn.BatchNorm1d(32)
        self.fc2 = nn.Linear(32, 8)
        self.dropout2 = nn.Dropout(dropout)
        self.bn2 = nn.BatchNorm1d(8)
        self.fc3 = nn.Linear(8, 2)

        # If initialization function provided, call it on the weights of the model
        if initialization is not None:
            initialization(self.fc1.weight)
            initialization(self.fc2.weight)
            initialization(self.fc3.weight)

    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(x)
        x = self.conv4(x)
        x = self.conv5(x)
        #x = self.conv6(x)
        
        x = x.flatten(1)
        
        x = self.bn0(x)
        x = self.dropout1(self.bn1(F.relu(self.fc1(x))))
        x = self.dropout2(self.bn2(F.relu(self.fc2(x))))
        x = self.fc3(x)
        return x

In [14]:
import torch.optim as optim
from tqdm import tqdm

loss_fn = nn.CrossEntropyLoss() # softmax + neg. log likelihood

def train_model(model, epochs=3, lr=0.001):
    optimizer = optim.Adam(model.parameters(), lr=lr)

    train_loss = []
    val_loss = []
    val_accuracy = []
    for epoch in range(epochs):
        model.train() # train mode (affects batchnorm layers:
                      # in the subsequent forward passes they'll
                      # exhibit 'train' behaviour, i.e. they'll
                      # normalize activations over batches)
        for i, (X, y) in enumerate(tqdm(trainloader)):
            X, y = X.to(device), y.to(device)

            pred = model(X)
            loss = loss_fn(pred, y)
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()
            print(y)
            train_loss.append(loss.item())

        model.eval() # test mode (affects batchnorm layers:
                     # in the subsequent forward passes they'll
                     # exhibit 'test' behaviour, i.e. they'll
                     # use the accumulated running statistics
                     # to normalize activations)
        epoch_losses = []
        epoch_accuracies = []
        with torch.no_grad(): # avoid calculating gradients during evaluation
            for X, y in valloader:
                X, y = X.to(device), y.to(device)

                pred = model(X)

                epoch_losses.append(loss_fn(pred, y).item())
                _, pred = torch.max(pred.data, 1) # pred = index of maximal output along axis=1

                epoch_accuracies.append(
                    (pred == y).to(torch.float32).mean().item()
                )
        val_loss.append(np.mean(epoch_losses))
        val_accuracy.append(np.mean(epoch_accuracies))

    return dict(
        train_loss=train_loss,
        val_loss=val_loss,
        val_accuracy=val_accuracy
    )

In [12]:
model = model(True, initialization=(lambda w: torch.nn.init.kaiming_normal_(w, nonlinearity='relu')), dropout=0.1)

In [15]:
train_model(model.to(device))

  0%|                                                                                  | 1/1200 [00:00<14:20,  1.39it/s]

tensor([0, 0, 1, 1, 1, 0, 1, 1, 0, 1], device='cuda:1')
tensor([0, 1, 0, 1, 0, 1, 1, 1, 0, 1], device='cuda:1')


  0%|▎                                                                                 | 5/1200 [00:01<03:15,  6.11it/s]

tensor([0, 1, 1, 0, 1, 1, 0, 1, 0, 0], device='cuda:1')
tensor([0, 1, 0, 1, 1, 1, 0, 0, 1, 1], device='cuda:1')
tensor([1, 0, 0, 0, 1, 1, 1, 1, 0, 0], device='cuda:1')
tensor([1, 1, 0, 0, 0, 1, 0, 1, 0, 1], device='cuda:1')


  1%|▌                                                                                 | 9/1200 [00:01<02:15,  8.76it/s]

...

tensor([0, 1, 0, 1, 1, 1, 1, 0, 1, 0], device='cuda:1')
tensor([0, 1, 0, 0, 0, 0, 0, 0, 1, 0], device='cuda:1')
tensor([1, 0, 0, 0, 0, 1, 1, 1, 1, 1], device='cuda:1')
tensor([0, 1, 1, 0, 0, 0, 1, 1, 0, 1], device='cuda:1')


  8%|██████▏                                                                          | 91/1200 [00:11<02:17,  8.09it/s]

tensor([1, 1, 0, 0, 1, 0, 1, 1, 1, 1], device='cuda:1')
tensor([0, 1, 0, 0, 1, 1, 1, 1, 0, 1], device='cuda:1')
tensor([0, 1, 0, 1, 0, 0, 0, 1, 1, 0], device='cuda:1')


  8%|██████▏                                                                          | 91/1200 [00:11<02:23,  7.72it/s]


KeyboardInterrupt: 

In [12]:
import PIL
import glob
import os

In [41]:
ds_test = datasets.ImageFolder('test', transform=transform)
ds_test

Dataset ImageFolder
    Number of datapoints: 16560
    Root location: test
    StandardTransform
Transform: Compose(
               Grayscale(num_output_channels=1)
               ToTensor()
               Normalize(mean=0.5, std=0.5)
               CenterCrop(size=(64, 64))
           )

In [25]:
testloader = torch.utils.data.DataLoader(ds_test, batch_size=10,
                                        shuffle=False, num_workers=2)

In [59]:
pred = []
with torch.no_grad():
    for X, y in testloader:
        i = i + 1
        X, y = X.to(device), y.to(device)
        pred_batch = (torch.sigmoid(model(X).data).cpu().numpy()[:,0])
        for i in np.arange(10):
            pred.append(pred_batch[i])

Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7fbb38442320>
Traceback (most recent call last):
  File "/mnt/project_mnt/teo_fs/anaconda3_ml/envs/ecanonero/lib/python3.7/site-packages/torch/utils/data/dataloader.py", line 1328, in __del__
    self._shutdown_workers()
  File "/mnt/project_mnt/teo_fs/anaconda3_ml/envs/ecanonero/lib/python3.7/site-packages/torch/utils/data/dataloader.py", line 1320, in _shutdown_workers
    if w.is_alive():
  File "/mnt/project_mnt/teo_fs/anaconda3_ml/envs/ecanonero/lib/python3.7/multiprocessing/process.py", line 151, in is_alive
    assert self._parent_pid == os.getpid(), 'can only test a child process'
AssertionError: Exception ignored in: can only test a child process
<function _MultiProcessingDataLoaderIter.__del__ at 0x7fbb38442320>
Exception ignored in: Traceback (most recent call last):
<function _MultiProcessingDataLoaderIter.__del__ at 0x7fbb38442320>  File "/mnt/project_mnt/teo_fs/anaconda3_ml/envs/ecanonero/lib/pyth

    assert self._parent_pid == os.getpid(), 'can only test a child process'
AssertionError: can only test a child process    
if w.is_alive():
Exception ignored in:   File "/mnt/project_mnt/teo_fs/anaconda3_ml/envs/ecanonero/lib/python3.7/multiprocessing/process.py", line 151, in is_alive
<function _MultiProcessingDataLoaderIter.__del__ at 0x7fbb38442320>
Traceback (most recent call last):
  File "/mnt/project_mnt/teo_fs/anaconda3_ml/envs/ecanonero/lib/python3.7/site-packages/torch/utils/data/dataloader.py", line 1328, in __del__
    assert self._parent_pid == os.getpid(), 'can only test a child process'
AssertionError: can only test a child process
Exception ignored in:     <function _MultiProcessingDataLoaderIter.__del__ at 0x7fbb38442320>self._shutdown_workers()

Traceback (most recent call last):
  File "/mnt/project_mnt/teo_fs/anaconda3_ml/envs/ecanonero/lib/python3.7/site-packages/torch/utils/data/dataloader.py", line 1320, in _shutdown_workers
  File "/mnt/project_mnt/teo_fs/ana

In [60]:
pred[:10]

[0.92770207,
 0.94809866,
 0.036010675,
 0.27968818,
 0.0367986,
 0.038422275,
 0.95455766,
 0.06915416,
 0.050439186,
 0.3717272]

In [61]:
print(len(pred))

16560


In [64]:
pred_part=[]
i=0
for filename in sorted(glob.glob('test/test/*.png')):
    base=os.path.basename(filename)
    pred_part.append([os.path.splitext(base)[0], pred[i]])
    i = i + 1

In [65]:
pred_part

[['49f983150b879b9e3a246a508bc0209326cf1b3a', 0.92770207],
 ['07d07cb77690518f977d47bc433fa8f27c92166b', 0.94809866],
 ['a9e10973b1754bd4b1755d0a8763f13cbc61e27f', 0.036010675],
 ['2c96cf26aaa68b08f3ec1d2c81ef975d6fc61802', 0.27968818],
 ['ea6e028d2751c51113cc3d76745e4e3c42b44ccb', 0.0367986],
 ['2af0cd43744b782634e4541774744311cd4b38b0', 0.038422275],
 ['2b3c644878162adcc7942d771e88d4bcec19bef7', 0.95455766],
 ['be451739adbfad6d44517c6abf39bd2e601a0846', 0.06915416],
 ['08a593cb53939253a981d34ad338d6aed8fc6bbf', 0.050439186],
 ['10ffba17c017915dd2decb632f983211400f6e75', 0.3717272],
 ['1647bf199e36fd69306cc42daf2ea6dd1371857a', 0.86245],
 ['9a3e73d316cb4294d32119a94f4ee08a38fbc798', 0.20071483],
 ['1cc99d8da8349e842a5d4d48a994bb168f86d646', 0.07326957],
 ['01c48580fbbad40dd12984fe77963d326550e164', 0.07307919],
 ['6ef3d4f703044d747b7f52e86d7f04370241b4a4', 0.09749442],
 ['c0b74f92ff3a19489f9c91c288d2a39d2d00c96c', 0.03724613],
 ['89286691cf9c24af194682c32d2aa8e949c98581', 0.9694821],


In [69]:
import pandas as pd 
pd.DataFrame(pred_part).to_csv("pred.csv")

In [71]:
df = pd.DataFrame(data=pred_part, columns=["id", "particle"])

In [72]:
df.head()

,id,particle
0,49f983150b879b9e3a246a508bc0209326cf1b3a,0.927702
1,07d07cb77690518f977d47bc433fa8f27c92166b,0.948099
2,a9e10973b1754bd4b1755d0a8763f13cbc61e27f,0.036011
3,2c96cf26aaa68b08f3ec1d2c81ef975d6fc61802,0.279688
4,ea6e028d2751c51113cc3d76745e4e3c42b44ccb,0.036799


In [74]:
df.to_csv("pred.csv", index=False)